In [2]:
#Professor's Code
import random
import string

s = string.ascii_lowercase + string.digits
n = 100200
payee_ids = [None]*n

# set the random seed, so results are replicable
random.seed( 1090 )

for i in range(n):
    payee_ids[i] = ''.join(random.choice(s) for _ in range(10))

# Using list comprehensions would be much more concise.
# But this is little bit hard to understand as it has two layers of list comprehensions
# payee_ids = [''.join(random.choice(s) for i in range(10)) for _ in range(n)]

# Exclude any possible duplicates (extremely unlikely, but possible)
payee_ids = list(set(payee_ids))[0:100000]

print(payee_ids[0:10])

['u38zmqzl0a', 'melctwvjke', 'twpbvbfo8w', 'o0500yx5ym', 'qrsnxjrl58', 'l4hf8rkurs', 'xczl4nl8wc', 'bdl6rsdquv', 'tjwycavt5m', 'ogzb0e5ajr']


In [3]:
#Professor's Code
# import datetime and timedelta classes from the datetime module
from datetime import datetime, timedelta

fDate = datetime(2023, 9, 1)
tDate = datetime(2023, 11, 25)

dd = (tDate - fDate).days
nn = round(len(payee_ids)*dd*2.5) # on average, each payee makes two and half trips.

# This line should be removed or commented out for the final run.
nn = 100 # this number is used to debug the code. 

# the time during the day does not really matter for OMNY rules. We just set it to a random number between 5 to 23.
payTime = [fDate + timedelta(days = random.uniform(0, dd), \
                             hours = random.choice(range(5,24)), \
                             minutes = random.choice(range(0, 60)), \
                             seconds=random.choice(range(0, 60))) for _ in range(nn)]

payees = random.choices(payee_ids, weights=[abs(random.normalvariate(0,1)) for _ in range(len(payee_ids))], k=nn)

simData = [list(x) for x in zip(payees, payTime)]
# Sort according to the time, from early to late.
simSData = sorted(simData, key= lambda z: z[1])

# Print the top 10 rows
for k in range(10):
    print()
    print("Index: {}, ID: {}, Time: {}".format(k, simSData[k][0], "" + simSData[k][1].strftime("%Y/%m/%d %I:%M%p, %A")))


Index: 0, ID: loyaxl46oh, Time: 2023/09/01 06:45PM, Friday

Index: 1, ID: 5i49cvo94d, Time: 2023/09/02 07:43PM, Saturday

Index: 2, ID: cs4rrqwa3e, Time: 2023/09/03 05:44AM, Sunday

Index: 3, ID: 87gmk55w19, Time: 2023/09/03 03:59PM, Sunday

Index: 4, ID: qsp83n14c1, Time: 2023/09/05 12:40AM, Tuesday

Index: 5, ID: gwglw2os5j, Time: 2023/09/06 06:44PM, Wednesday

Index: 6, ID: kbiawo6stu, Time: 2023/09/07 05:54AM, Thursday

Index: 7, ID: h6864sswel, Time: 2023/09/07 08:20AM, Thursday

Index: 8, ID: uw5jm18cgi, Time: 2023/09/07 06:25PM, Thursday

Index: 9, ID: nrm8c9lk0y, Time: 2023/09/08 04:27AM, Friday


In [4]:
#Professor's Code
for k in range(10):
    print()
    print("Index: {}, ID: {}, Time: {}, Charge: ${}".format(k, 
        simSData[k][0], 
        "" + simSData[k][1].strftime("%Y/%m/%d %I:%M%p, %A"), 
        random.choice([2.90, 0.00]) # This random value should be replaced by values from your algorithm.
    ))


Index: 0, ID: loyaxl46oh, Time: 2023/09/01 06:45PM, Friday, Charge: $0.0

Index: 1, ID: 5i49cvo94d, Time: 2023/09/02 07:43PM, Saturday, Charge: $0.0

Index: 2, ID: cs4rrqwa3e, Time: 2023/09/03 05:44AM, Sunday, Charge: $2.9

Index: 3, ID: 87gmk55w19, Time: 2023/09/03 03:59PM, Sunday, Charge: $0.0

Index: 4, ID: qsp83n14c1, Time: 2023/09/05 12:40AM, Tuesday, Charge: $2.9

Index: 5, ID: gwglw2os5j, Time: 2023/09/06 06:44PM, Wednesday, Charge: $2.9

Index: 6, ID: kbiawo6stu, Time: 2023/09/07 05:54AM, Thursday, Charge: $2.9

Index: 7, ID: h6864sswel, Time: 2023/09/07 08:20AM, Thursday, Charge: $2.9

Index: 8, ID: uw5jm18cgi, Time: 2023/09/07 06:25PM, Thursday, Charge: $0.0

Index: 9, ID: nrm8c9lk0y, Time: 2023/09/08 04:27AM, Friday, Charge: $0.0


In [ ]:
# My addon using a Boolean and a Double Ended Queue
from collections import deque
from datetime import datetime, timedelta

# Function to determine whether to charge a fare or provide a free ride
def determine_fare(transactions):
    payee_history = {}  # Dictionary to store the last 7 days of transactions for each payee
    fares = []  # List to store the results (payee_id, timestamp, fare decision)
    
    for payee_id, pay_time in transactions:
        if payee_id not in payee_history:
            payee_history[payee_id] = deque()  # Initialize a deque for the payee to store timestamps
        
        # Remove trips that are older than 7 days from the deque
        while payee_history[payee_id] and pay_time - payee_history[payee_id][0] > timedelta(days=7):
            payee_history[payee_id].popleft()
        
        # Check if the payee has already reached 12 paid trips in the last 7 days
        is_free = len(payee_history[payee_id]) >= 12
        fares.append((payee_id, pay_time, not is_free))  # True if chargeable, False if free
        
        if not is_free:
            payee_history[payee_id].append(pay_time)  # Store the current transaction timestamp
    
    return fares  # Return the list of transactions with applicable fares

# Example usage with simulated data
simulated_fares = determine_fare(simSData)

# Print the top 10 results
for k in range(10):
    print("Index: {}, ID: {}, Time: {}, Charge: {}".format(
        k,
        simulated_fares[k][0],  # Payee ID
        simulated_fares[k][1].strftime("%Y/%m/%d %I:%M%p, %A"),  # Payment timestamp
        simulated_fares[k][2]  # Boolean value (True = Charged, False = Free)
    ))


Index: 0, ID: lamj6h3sot, Time: 2023/09/01 06:45PM, Friday, Charge: True
Index: 1, ID: 9v3r8fcuy1, Time: 2023/09/02 07:43PM, Saturday, Charge: True
Index: 2, ID: eeec21otxb, Time: 2023/09/03 05:44AM, Sunday, Charge: True
Index: 3, ID: 78bf8nfkgn, Time: 2023/09/03 03:59PM, Sunday, Charge: True
Index: 4, ID: e6p5z72rb5, Time: 2023/09/05 12:40AM, Tuesday, Charge: True
Index: 5, ID: vti6m0aoz1, Time: 2023/09/06 06:44PM, Wednesday, Charge: True
Index: 6, ID: wbx919zplh, Time: 2023/09/07 05:54AM, Thursday, Charge: True
Index: 7, ID: fql3qs0wtx, Time: 2023/09/07 08:20AM, Thursday, Charge: True
Index: 8, ID: puu0ln55b4, Time: 2023/09/07 06:25PM, Thursday, Charge: True
Index: 9, ID: v26li4i9h8, Time: 2023/09/08 04:27AM, Friday, Charge: True
